# Representation III
## Practical Exercises

TO RUN THE NEURAL NETWORKS EXERCISES YOU WILL NEED TO HAVE KERAS (https://keras.io/) INSTALLED.

### Exercise 3-3
We will look at the **Covertype** dataset using boosted decision trees and boosted linear SVM. 
The dataset predicts forest cover type from GIS features: https://archive.ics.uci.edu/ml/datasets/Covertype

The aim is to distinguish lodgepole pine from other forest types (spruce/fir, Ponderosa pine, cottonwood/willow, aspen, Douglas-fir). The features are:

* Elevation / quantitative /meters / Elevation in meters 
* Aspect / quantitative / azimuth / Aspect in degrees azimuth 
* Slope / quantitative / degrees / Slope in degrees 
* Horizontal_Distance_To_Hydrology / quantitative / meters / Horz Dist to nearest surface water features 
* Vertical_Distance_To_Hydrology / quantitative / meters / Vert Dist to nearest surface water features 
* Horizontal_Distance_To_Roadways / quantitative / meters / Horz Dist to nearest roadway 
* Hillshade_9am / quantitative / 0 to 255 index / Hillshade index at 9am, summer solstice 
* Hillshade_Noon / quantitative / 0 to 255 index / Hillshade index at noon, summer soltice 
* Hillshade_3pm / quantitative / 0 to 255 index / Hillshade index at 3pm, summer solstice 
* Horizontal_Distance_To_Fire_Points / quantitative / meters / Horz Dist to nearest wildfire ignition points 
* Wilderness_Area (4 binary columns) / qualitative / 0 (absence) or 1 (presence) / Wilderness area designation 
* Soil_Type (40 binary columns) / qualitative / 0 (absence) or 1 (presence) / Soil Type designation 
* Cover_Type (7 types) / integer / 1 to 7 / Forest Cover Type designation

Run the experiment. Then comment on the suitability of decision trees and SVM as weak learners for this dataset.

In [ ]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

X, Y = load_svmlight_file("covertype.txt")

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.9, random_state=42)

clf = DecisionTreeClassifier(random_state=42, max_depth=20)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_train)
accuracy = accuracy_score(y_train, predict)
print("DT training accuracy: " + "{0:.2f}".format(accuracy))
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("DT test accuracy: " + "{0:.2f}".format(accuracy))

clf = svm.LinearSVC(loss='hinge', C=1, random_state = 42)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_train)
accuracy = accuracy_score(y_train, predict)
print("SVM training accuracy: " + "{0:.2f}".format(accuracy))
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("SVM test accuracy: " + "{0:.2f}".format(accuracy))

clf = AdaBoostClassifier(DecisionTreeClassifier(random_state=42, max_depth=20),
                         n_estimators=10,random_state=42, algorithm='SAMME')
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_train)
accuracy = accuracy_score(y_train, predict)
print("Boosted DT training accuracy: " + "{0:.2f}".format(accuracy))
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("Boosted DT test accuracy: " + "{0:.2f}".format(accuracy))

clf = AdaBoostClassifier(svm.LinearSVC(loss='hinge', C=1, random_state = 42),
                         n_estimators=10,random_state=42, algorithm='SAMME')
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_train)
accuracy = accuracy_score(y_train, predict)
print("Boosted SVM training accuracy: " + "{0:.2f}".format(accuracy))
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("Boosted SVM test accuracy: " + "{0:.2f}".format(accuracy))

### Exercise 3-7
  The earlier layers of deep neural networks can be considered as features. Deeper units represent more complex features composed from simpler features. These features are learned as part of supervised learning. In this exercise, we consider whether features that are learned for one task can be used for another task. This is sometimes called transfer learning.
  
  A convolutional neural network with a single convolutional layer of 32 filters and a fully connected layer with 128 units is trained on the MNIST digit classification dataset for digits 0 to 4. We then freeze the convolutional layer with the trained weights, and train the rest of the network with digits 5 to 9. Call this the transfer scheme.
  
  Compare the performance of the transfer scheme with training from scratch for 300, 1000, 3000, and 10,000 training examples for the digits 5 to 9.

Comment on the results.

In [1]:
# Modified from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py
'''Transfer learning toy example:
1- Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
2- Freeze convolutional layers and fine-tune dense layers
   for the classification of digits [5..9].
Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python mnist_transfer_cnn.py
Get to 99.8% test accuracy after 5 epochs
for the first five digits classifier
and 99.2% for the last five digits after transfer + fine-tuning.
'''

from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)


def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])


# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

train_sizes = [300, 1000, 3000, 10000]

# Train model from scratch for 5-digit classification [5..9]
for i in range(4):
    model = Sequential(feature_layers + classification_layers)
    train_model(model,
            (x_train_gte5[0:train_sizes[i]], y_train_gte5[0:train_sizes[i]]),
            (x_test_gte5[0:train_sizes[i]], y_test_gte5[0:train_sizes[i]]), num_classes)

# Whole training dataset
model = Sequential(feature_layers + classification_layers)
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

# train model for 5-digit classification [0..4]
model = Sequential(feature_layers + classification_layers)
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

# Now train with feature layers frozen for 5-digit classification [5..9]                                                       
for i in range(4):
    # freeze feature layers and rebuild model
    for l in feature_layers:
        l.trainable = False                                    
    train_model(model,
            (x_train_gte5[0:train_sizes[i]], y_train_gte5[0:train_sizes[i]]),
            (x_test_gte5[0:train_sizes[i]], y_test_gte5[0:train_sizes[i]]), num_classes) 

# Whole training dataset
for l in feature_layers:
    l.trainable = False
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)                                                       



Using TensorFlow backend.


11493376/11490434 [==============================] - 14s 1us/step
x_train shape: (300, 28, 28, 1)
300 train samples
300 test samples
Train on 300 samples, validate on 300 samples
Epoch 1/5
300/300 [==============================] - 0s 2ms/step - loss: 1.5381 - accuracy: 0.3667 - val_loss: 1.4112 - val_accuracy: 0.3600
Epoch 2/5
300/300 [==============================] - 0s 894us/step - loss: 1.2456 - accuracy: 0.5867 - val_loss: 1.1245 - val_accuracy: 0.6000
Epoch 3/5
300/300 [==============================] - 0s 835us/step - loss: 1.0402 - accuracy: 0.6000 - val_loss: 0.9693 - val_accuracy: 0.6367
Epoch 4/5
300/300 [==============================] - 0s 886us/step - loss: 0.6937 - accuracy: 0.7667 - val_loss: 0.7828 - val_accuracy: 0.7067
Epoch 5/5
300/300 [==============================] - 0s 891us/step - loss: 0.6681 - accuracy: 0.8167 - val_loss: 0.7506 - val_accuracy: 0.7033
Training time: 0:00:01.972580
Test score: 0.7506122608979543
Test accuracy: 0.70333331823349
x_train shape: 

Epoch 4/5
3000/3000 [==============================] - 1s 362us/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.0424 - val_accuracy: 0.9900
Epoch 5/5
3000/3000 [==============================] - 1s 378us/step - loss: 0.0212 - accuracy: 0.9943 - val_loss: 0.0376 - val_accuracy: 0.9910
Training time: 0:00:06.176544
Test score: 0.03756344548520671
Test accuracy: 0.9909999966621399
x_train shape: (10000, 28, 28, 1)
10000 train samples
4861 test samples
Train on 10000 samples, validate on 4861 samples
Epoch 1/5
10000/10000 [==============================] - 3s 309us/step - loss: 0.0352 - accuracy: 0.9898 - val_loss: 0.0236 - val_accuracy: 0.9934
Epoch 2/5
10000/10000 [==============================] - 3s 299us/step - loss: 0.0224 - accuracy: 0.9929 - val_loss: 0.0231 - val_accuracy: 0.9934
Epoch 3/5
10000/10000 [==============================] - 3s 326us/step - loss: 0.0208 - accuracy: 0.9927 - val_loss: 0.0204 - val_accuracy: 0.9940
Epoch 4/5
10000/10000 [=============================

### Exercise 3-8
In the experiment, we will use a recurrent neural network to add two numbers represented as strings. For example the output of "535+61" should be "596". As the code takes a long time to run, you can run it in your own time.

The code suggests that inverting the input digit sequences may make the problem easier to learn. Suggest why that may be the case.

In [2]:
# Code from https://github.com/fchollet/keras/blob/master/examples/addition_rnn.py
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=" ")
        else:
            print(colors.fail + '☒' + colors.close, end=" ")
        print(guess)
        print('---')

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_5 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_______

45000/45000 [==============================] - 7s 163us/step - loss: 0.7553 - accuracy: 0.7306 - val_loss: 0.7516 - val_accuracy: 0.7254
Q 35+971 
T 1006
☒ 1000
---
Q 809+67 
T 876 
☒ 877 
---
Q 53+978 
T 1031
☒ 1021
---
Q 622+574
T 1196
☒ 1197
---
Q 69+860 
T 929 
☒ 925 
---
Q 535+680
T 1215
☒ 1217
---
Q 989+396
T 1385
☒ 1375
---
Q 534+47 
T 581 
☒ 577 
---
Q 43+740 
T 783 
☒ 773 
---
Q 73+280 
T 353 
☒ 350 
---

--------------------------------------------------
Iteration 13
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 167us/step - loss: 0.7053 - accuracy: 0.7507 - val_loss: 0.6901 - val_accuracy: 0.7535
Q 903+53 
T 956 
☒ 957 
---
Q 478+570
T 1048
☒ 1059
---
Q 781+177
T 958 
☒ 959 
---
Q 491+841
T 1332
☒ 1333
---
Q 981+1  
T 982 
☒ 983 
---
Q 187+154
T 341 
☒ 345 
---
Q 20+17  
T 37  
☒ 47  
---
Q 10+77  
T 87  
☑ 87  
---
Q 684+76 
T 760 
☑ 760 
---
Q 0+508  
T 508 
☒ 509 
---

-----------------------------------------

45000/45000 [==============================] - 7s 160us/step - loss: 0.0458 - accuracy: 0.9946 - val_loss: 0.0526 - val_accuracy: 0.9906
Q 11+741 
T 752 
☑ 752 
---
Q 452+71 
T 523 
☑ 523 
---
Q 75+37  
T 112 
☑ 112 
---
Q 531+84 
T 615 
☑ 615 
---
Q 53+594 
T 647 
☑ 647 
---
Q 690+86 
T 776 
☑ 776 
---
Q 704+78 
T 782 
☑ 782 
---
Q 47+244 
T 291 
☑ 291 
---
Q 100+908
T 1008
☑ 1008
---
Q 701+96 
T 797 
☑ 797 
---

--------------------------------------------------
Iteration 26
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 160us/step - loss: 0.0529 - accuracy: 0.9895 - val_loss: 0.0430 - val_accuracy: 0.9930
Q 327+510
T 837 
☑ 837 
---
Q 27+596 
T 623 
☑ 623 
---
Q 419+471
T 890 
☑ 890 
---
Q 42+982 
T 1024
☑ 1024
---
Q 177+76 
T 253 
☑ 253 
---
Q 51+171 
T 222 
☑ 222 
---
Q 72+384 
T 456 
☑ 456 
---
Q 346+765
T 1111
☑ 1111
---
Q 521+7  
T 528 
☑ 528 
---
Q 164+538
T 702 
☑ 702 
---

-----------------------------------------

45000/45000 [==============================] - 7s 159us/step - loss: 0.0083 - accuracy: 0.9995 - val_loss: 0.0104 - val_accuracy: 0.9989
Q 358+336
T 694 
☑ 694 
---
Q 28+173 
T 201 
☑ 201 
---
Q 829+792
T 1621
☑ 1621
---
Q 41+623 
T 664 
☑ 664 
---
Q 96+749 
T 845 
☑ 845 
---
Q 8+744  
T 752 
☑ 752 
---
Q 371+912
T 1283
☑ 1283
---
Q 883+761
T 1644
☑ 1644
---
Q 60+682 
T 742 
☑ 742 
---
Q 62+46  
T 108 
☑ 108 
---

--------------------------------------------------
Iteration 39
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 157us/step - loss: 0.0069 - accuracy: 0.9997 - val_loss: 0.0098 - val_accuracy: 0.9988
Q 245+769
T 1014
☑ 1014
---
Q 188+708
T 896 
☑ 896 
---
Q 664+9  
T 673 
☑ 673 
---
Q 16+902 
T 918 
☑ 918 
---
Q 318+513
T 831 
☑ 831 
---
Q 271+937
T 1208
☑ 1208
---
Q 486+3  
T 489 
☑ 489 
---
Q 36+35  
T 71  
☑ 71  
---
Q 92+623 
T 715 
☑ 715 
---
Q 52+16  
T 68  
☑ 68  
---

-----------------------------------------

45000/45000 [==============================] - 8s 177us/step - loss: 0.0159 - accuracy: 0.9962 - val_loss: 0.0067 - val_accuracy: 0.9990
Q 71+7   
T 78  
☑ 78  
---
Q 982+260
T 1242
☑ 1242
---
Q 596+176
T 772 
☑ 772 
---
Q 887+70 
T 957 
☑ 957 
---
Q 379+88 
T 467 
☑ 467 
---
Q 25+966 
T 991 
☑ 991 
---
Q 32+93  
T 125 
☑ 125 
---
Q 709+803
T 1512
☑ 1512
---
Q 132+776
T 908 
☑ 908 
---
Q 269+13 
T 282 
☑ 282 
---

--------------------------------------------------
Iteration 52
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 178us/step - loss: 0.0034 - accuracy: 0.9999 - val_loss: 0.0052 - val_accuracy: 0.9994
Q 24+860 
T 884 
☑ 884 
---
Q 105+510
T 615 
☑ 615 
---
Q 755+833
T 1588
☑ 1588
---
Q 588+1  
T 589 
☑ 589 
---
Q 544+76 
T 620 
☑ 620 
---
Q 82+143 
T 225 
☑ 225 
---
Q 65+885 
T 950 
☑ 950 
---
Q 811+23 
T 834 
☑ 834 
---
Q 77+15  
T 92  
☑ 92  
---
Q 338+90 
T 428 
☑ 428 
---

-----------------------------------------

...

45000/45000 [==============================] - 8s 170us/step - loss: 3.2311e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 0.9998
Q 378+860
T 1238
☑ 1238
---
Q 231+6  
T 237 
☑ 237 
---
Q 728+98 
T 826 
☑ 826 
---
Q 986+5  
T 991 
☑ 991 
---
Q 154+117
T 271 
☑ 271 
---
Q 28+153 
T 181 
☑ 181 
---
Q 79+968 
T 1047
☑ 1047
---
Q 379+12 
T 391 
☑ 391 
---
Q 377+79 
T 456 
☑ 456 
---
Q 771+911
T 1682
☑ 1682
---

--------------------------------------------------
Iteration 143
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 159us/step - loss: 2.9136e-04 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 0.9998
Q 83+977 
T 1060
☑ 1060
---
Q 506+706
T 1212
☑ 1212
---
Q 9+33   
T 42  
☑ 42  
---
Q 33+94  
T 127 
☑ 127 
---
Q 45+899 
T 944 
☑ 944 
---
Q 473+575
T 1048
☑ 1048
---
Q 194+74 
T 268 
☑ 268 
---
Q 489+90 
T 579 
☑ 579 
---
Q 9+156  
T 165 
☑ 165 
---
Q 35+20  
T 55  
☑ 55  
---

--------------------------------

45000/45000 [==============================] - 8s 169us/step - loss: 3.8677e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 0.9998
Q 134+58 
T 192 
☑ 192 
---
Q 721+76 
T 797 
☑ 797 
---
Q 768+51 
T 819 
☑ 819 
---
Q 320+41 
T 361 
☑ 361 
---
Q 24+63  
T 87  
☑ 87  
---
Q 723+355
T 1078
☑ 1078
---
Q 50+58  
T 108 
☑ 108 
---
Q 404+61 
T 465 
☑ 465 
---
Q 553+14 
T 567 
☑ 567 
---
Q 1+151  
T 152 
☑ 152 
---

--------------------------------------------------
Iteration 156
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 170us/step - loss: 3.2318e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 0.9998
Q 162+43 
T 205 
☑ 205 
---
Q 140+406
T 546 
☑ 546 
---
Q 6+403  
T 409 
☑ 409 
---
Q 729+599
T 1328
☑ 1328
---
Q 233+19 
T 252 
☑ 252 
---
Q 912+494
T 1406
☑ 1406
---
Q 57+76  
T 133 
☑ 133 
---
Q 27+47  
T 74  
☑ 74  
---
Q 693+41 
T 734 
☑ 734 
---
Q 740+91 
T 831 
☑ 831 
---

--------------------------------

45000/45000 [==============================] - 8s 167us/step - loss: 2.4342e-04 - accuracy: 1.0000 - val_loss: 7.9782e-04 - val_accuracy: 0.9999
Q 174+26 
T 200 
☑ 200 
---
Q 117+97 
T 214 
☑ 214 
---
Q 17+396 
T 413 
☑ 413 
---
Q 874+988
T 1862
☑ 1862
---
Q 273+144
T 417 
☑ 417 
---
Q 35+35  
T 70  
☑ 70  
---
Q 457+695
T 1152
☑ 1152
---
Q 526+26 
T 552 
☑ 552 
---
Q 83+634 
T 717 
☑ 717 
---
Q 8+744  
T 752 
☑ 752 
---

--------------------------------------------------
Iteration 169
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 166us/step - loss: 2.2044e-04 - accuracy: 1.0000 - val_loss: 7.1092e-04 - val_accuracy: 0.9999
Q 834+56 
T 890 
☑ 890 
---
Q 22+593 
T 615 
☑ 615 
---
Q 789+764
T 1553
☑ 1553
---
Q 990+24 
T 1014
☑ 1014
---
Q 941+42 
T 983 
☑ 983 
---
Q 7+637  
T 644 
☑ 644 
---
Q 817+20 
T 837 
☑ 837 
---
Q 2+604  
T 606 
☑ 606 
---
Q 840+0  
T 840 
☑ 840 
---
Q 933+83 
T 1016
☑ 1016
---

------------------------

45000/45000 [==============================] - 7s 160us/step - loss: 1.7028e-04 - accuracy: 1.0000 - val_loss: 7.5506e-04 - val_accuracy: 0.9998
Q 583+592
T 1175
☑ 1175
---
Q 41+361 
T 402 
☑ 402 
---
Q 2+334  
T 336 
☑ 336 
---
Q 318+104
T 422 
☑ 422 
---
Q 630+905
T 1535
☑ 1535
---
Q 5+683  
T 688 
☑ 688 
---
Q 8+652  
T 660 
☑ 660 
---
Q 875+66 
T 941 
☑ 941 
---
Q 5+211  
T 216 
☑ 216 
---
Q 795+44 
T 839 
☑ 839 
---

--------------------------------------------------
Iteration 182
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 168us/step - loss: 1.5649e-04 - accuracy: 1.0000 - val_loss: 6.9167e-04 - val_accuracy: 0.9998
Q 595+83 
T 678 
☑ 678 
---
Q 927+73 
T 1000
☑ 1000
---
Q 489+84 
T 573 
☑ 573 
---
Q 8+594  
T 602 
☑ 602 
---
Q 701+8  
T 709 
☑ 709 
---
Q 63+164 
T 227 
☑ 227 
---
Q 570+159
T 729 
☑ 729 
---
Q 356+95 
T 451 
☑ 451 
---
Q 256+160
T 416 
☑ 416 
---
Q 446+72 
T 518 
☑ 518 
---

------------------------

45000/45000 [==============================] - 7s 155us/step - loss: 0.0295 - accuracy: 0.9921 - val_loss: 0.0036 - val_accuracy: 0.9991
Q 974+723
T 1697
☑ 1697
---
Q 62+10  
T 72  
☑ 72  
---
Q 23+985 
T 1008
☑ 1008
---
Q 28+72  
T 100 
☑ 100 
---
Q 399+6  
T 405 
☑ 405 
---
Q 32+284 
T 316 
☑ 316 
---
Q 390+4  
T 394 
☑ 394 
---
Q 466+3  
T 469 
☑ 469 
---
Q 90+413 
T 503 
☑ 503 
---
Q 422+118
T 540 
☑ 540 
---

--------------------------------------------------
Iteration 195
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 157us/step - loss: 8.4916e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9997
Q 34+90  
T 124 
☑ 124 
---
Q 549+45 
T 594 
☑ 594 
---
Q 11+200 
T 211 
☑ 211 
---
Q 74+417 
T 491 
☑ 491 
---
Q 671+14 
T 685 
☑ 685 
---
Q 52+357 
T 409 
☑ 409 
---
Q 130+220
T 350 
☑ 350 
---
Q 10+77  
T 87  
☑ 87  
---
Q 95+646 
T 741 
☑ 741 
---
Q 633+431
T 1064
☑ 1064
---

------------------------------------